María Carelia Ibarra Vásquez

A01251463

Modelación de sistemas multiagentes con gráficas computacionales TC2008B.101

18 de agosto del 2022

# M1. Actividad

### Reporte

Las aspiradoras robóticas, a comparación con las aspiradoras comúnes, se destacan por su capacidad de limpiar y navegar por sí mismas. Estos robots tienen diferentes sensores que les ayudan a detectar paredes, escaleras, o cualquier otro obstáculo que impida su camino. 

El programa a continuación simula estos robots dentro de un cuarto con áreas sucias que el robot debe limpiar. El resultado del código varia dependiendo de varios factores, por ejemplo: el área del grid, la cantidad de casillas sucias, la cantidad de robots en la escena, los movimientos que el robot puede realizar, y la cantidad de steps que se pueden hacer en una corrida.

da Costa, C. (2022). *How Do Robot Vacuums Work?*. https://www.gadgetreview.com/how-do-robot-vacuums-work

### Código

In [1]:
# Model design
import agentpy as ap
import random

MOVES = [(1,0), (0,1), (-1,0), (0,-1)]

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [4]:
class RoomModel(ap.Model):
    
    def setup(self):
       
        # Create agents (cleaning robots) 
        self.robot = ap.AgentList(self, 5);
        self.dirty_room = ap.AgentList(self, 200);
        

        # Create grid (rooms)
        self.room = ap.Grid(self, [50,60], track_empty=True)     
        self.room.add_agents(self.robot, [(1,1)]*10) 
        self.room.add_agents(self.dirty_room, random=True)

        
        # Start cleaning process from the left side of the grid
        self.robot.type_agent = 0
        self.dirty_room.type_agent = 1
        
    def step(self):
      
        robots = self.robot

        # Spread cleaning robots 
        for robot in robots:
            for neighbor in self.room.neighbors(robot):
                if neighbor.type_agent == 1:
                    neighbor.type_agent = 2 
                    break
            else:
              self.room.move_by(robot, random.choice(MOVES))   
        trash_pending = self.dirty_room.select(self.dirty_room.type_agent == 1)
        # Stop simulation if no dirty room is left
        if len(trash_pending) == 0: 
            self.stop()
   
    def end(self):
        
        # Document a measure at the end of the simulation
        dirty_room = len(self.room.select(self.room.type_agent == 2))
        self.report('Percentage of dirty rooms', 
                    dirty_room / len(self.dirty_room))

In [3]:
# Define parameters

parameters = {
    #'Dirty rooms': 0.6, # Percentage of grid
    #'size': 50, # Height and length of the grid
    'steps': 100,
}

In [5]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('type_agent')
    color_dict = {0:'#63666A', 1:'#CCB389', 2:'#D0F1EC', None:'#FFFDFA'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of robots cleaning rooms\n"
                 f"Time-step: {model.t}, Rooms left: "
                 f"{len(model.dirty_room.select(model.dirty_room.type_agent == 1))}")

fig, ax = plt.subplots() 
model = RoomModel({'steps': 100})
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

In [6]:
# Prepare parameter sample
parameters = {
    'Dirty rooms': ap.Range(0.2, 0.6), 
    #'size': 50,
}

sample = ap.Sample(parameters, n=30)

In [7]:
# Perform experiment
exp = ap.Experiment(RoomModel, sample, iterations=40)
results = exp.run()

Scheduled runs: 1200


AttributeError: Grid (Obj 206) has no attribute 'select'.

In [8]:
# Plot sensitivity
sns.set_theme()
sns.lineplot(
    data=results.arrange_reporters(), 
    x='Dirty rooms', 
    y='Percentage of clean rooms'
);

NameError: name 'results' is not defined